# Interactive Plot
This notebook contains our interactive, animated chart, created in Plotly. The x-axis is the song density, the y-axis is the similar verse score, and the size of each circle is how many unique words are used. Each genre is denoted by a different color, as shown in the legend.

In [1]:
import json
import pandas as pd

# read in the dataframe
df = pd.read_json('lyrics_dataframe.json')

In [ ]:
# This code was adapated heavily from the Plotly Animation tutorial, located here:
# https://plot.ly/python/animations/

import plotly.graph_objects as go

dataset = df.dropna()

years = [str(i) for i in range(1960, 2019)]

# make list of genres
genres = ['Pop', 'Rap', 'Rock', 'Other']
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# x axis: density, y axis: similar verses score, size: number of unique words in the song
fig_dict["layout"]["xaxis"] = {"range": [0, 7], "title": "Song Density"}
# fig_dict["layout"]["yaxis"] = {"title": "Similar Verses Score"}
fig_dict["layout"]["yaxis"] = {"range": [-1, 2.1], "title": "Similar Verses Score", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 400,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": "1960",
    "plotlycommand": "animate",
    "values": years,
    "visible": True
}

In [ ]:
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

In [ ]:
sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
year = 1960
for genre in genres:
    dataset_by_year = dataset[dataset["year"] == year]
    dataset_by_year_and_genre = dataset_by_year[dataset_by_year["genre"] == genre]

    data_dict = {
        "x": list(dataset_by_year_and_genre["density"]),
        "y": list(dataset_by_year_and_genre["similar_verses_score"]),
        "mode": "markers",
        # show the artist and song name
        "text": [title + " - " + artist for title,artist in zip(dataset_by_year_and_genre['title'], dataset_by_year_and_genre['artist'])],
        "marker": {
            "sizemode": "area",
            "sizeref": df["num_unique_words"].max() / 15 ** 2,
            "size": list(dataset_by_year_and_genre["num_unique_words"])
        },
        "name": genre
    }
    fig_dict["data"].append(data_dict)

In [2]:
# make frames
for year in years:
    frame = {"data": [], "name": year}
    for genre in genres:
        dataset_by_year = dataset[dataset["year"] == int(year)]
        dataset_by_year_and_genre = dataset_by_year[
            dataset_by_year["genre"] == genre]
        data_dict = {
            "x": list(dataset_by_year_and_genre["density"]),
            "y": list(dataset_by_year_and_genre["similar_verses_score"]),
            "mode": "markers",
            # show the artist and song name
            "text": [title + " - " + artist for title,artist in zip(dataset_by_year_and_genre['title'],dataset_by_year_and_genre['artist'])],
            "marker": {
                "sizemode": "area",
                "sizeref": df["num_unique_words"].max() / 15 ** 2,
                "size": list(dataset_by_year_and_genre["num_unique_words"])
            },
            "name": genre
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [year],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": year,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()